In [1]:
!pip install adversarial-robustness-toolbox

In [2]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [3]:
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescent
from art.attacks.evasion import BasicIterativeMethod
from art.estimators.classification import KerasClassifier


In [4]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [5]:
cd /gdrive/MyDrive/Net4011_Project_Part1

/gdrive/MyDrive/Net4011_Project_Part1


In [6]:
import numpy as np
import pandas as pd

#Load the CSV file as a dataframe
dataset_train = pd.read_csv("UNSW_2018_IoT_Botnet_Final_Training.csv")
dataset_test = pd.read_csv("UNSW_2018_IoT_Botnet_Final_Testing.csv")

dataset_train.head()


,pkSeqID,proto,saddr,sport,daddr,dport,seq,stddev,N_IN_Conn_P_SrcIP,min,state_number,mean,N_IN_Conn_P_DstIP,drate,srate,max,attack,category,subcategory
0,3142762,udp,192.168.100.150,6551,192.168.100.3,80,251984,1.900363,100,0.000000,4,2.687519,100,0.0,0.494549,4.031619,1,DDoS,UDP
1,2432264,tcp,192.168.100.150,5532,192.168.100.3,80,256724,0.078003,38,3.856930,3,3.934927,100,0.0,0.256493,4.012924,1,DDoS,TCP
2,1976315,tcp,192.168.100.147,27165,192.168.100.3,80,62921,0.268666,100,2.974100,3,3.341429,100,0.0,0.294880,3.609205,1,DDoS,TCP
3,1240757,udp,192.168.100.150,48719,192.168.100.3,80,99168,1.823185,63,0.000000,4,3.222832,63,0.0,0.461435,4.942302,1,DoS,UDP
4,3257991,udp,192.168.100.147,22461,192.168.100.3,80,105063,0.822418,100,2.979995,4,3.983222,100,0.0,1.002999,4.994452,1,DDoS,UDP


In [7]:
#Extract the features and the target from the dataframe
x_train = dataset_train.iloc[:,6:-3].values
x_test = dataset_test.iloc[:,6:-3].values
y_train = dataset_train.iloc[:,-2:-1].values
y_test = dataset_test.iloc[:,-2:-1].values

#Normalize the features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

#Encode the target (from string to array)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

#Convert labels to categorical variables
import keras
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


((2934817, 10), (733705, 10), (2934817, 5), (733705, 5))

In [8]:
#Define some variables
max_input = x_train.shape[1]
batch_size = 512
epochs = 10
plot = True

#Import Machine Learning Libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


"""Generic function to create a fully-connected neural network.

    # Arguments
        n_dense: int > 0. Number of dense layers.
        dense_units: int > 0. Number of dense units per layer.
        dropout: keras.layers.Layer. A dropout layer to apply.
        dropout_rate: 0 <= float <= 1. The rate of dropout.
        kernel_initializer: str. The initializer for the weights.
        optimizer: str/keras.optimizers.Optimizer. The optimizer to use.
        num_classes: int > 0. The number of classes to predict.
        max_input: int > 0. The maximum number of input features.

    # Returns
        A Keras model instance (compiled).
    """

def create_network(n_dense=1,
                   dense_units=6,
                   activation= 'relu',
                   dropout=Dropout,
                   dropout_rate=0.1,
                   kernel_initializer='glorot_uniform',
                   optimizer='adam',
                   num_classes=1,
                   max_input=x_train.shape[1]):
      #Layer 1
      # Initialize the model as a sequential one, which allows us to build the model layer by layer.
      model = Sequential()

      # Add the first dense layer with specified units and input shape. This layer is fully connected.
      # kernel_initializer sets the method for weight initialization.
      model.add(Dense(dense_units, input_shape=(max_input,),
                      kernel_initializer=kernel_initializer))
      # Add an activation function to the layer for introducing non-linearity.
      model.add(Activation(activation))
      # Add dropout to prevent overfitting, with the specified rate.
      model.add(dropout(dropout_rate))


      #Layer 2 to n-1
      # Add additional dense layers, if n_dense > 1. Each layer has the same number of units,
      # uses the same activation function, and includes dropout.
      for i in range(n_dense - 1):
          model.add(Dense(dense_units, kernel_initializer=kernel_initializer))
          model.add(Activation(activation))
          model.add(dropout(dropout_rate))

      #Layer n
      # Final layer: this is the output layer that has 'num_classes' units, one for each class.
      # For binary classification, 'num_classes' would typically be 1 (with 'sigmoid' activation).
      # For multi-class classification, it should match the number of classes and use 'softmax' activation to output probabilities.
      model.add(Dense(num_classes))
      model.add(Activation('softmax'))

      # Compile the model with the specified loss function, optimizer, and metrics to monitor.
      # 'categorical_crossentropy' is used for multi-class classification problems.
      model.compile(loss='categorical_crossentropy',
                    optimizer=optimizer,
                    metrics=['accuracy'])
      return model

#Overide any of the hyperparameters
network1 = {
    'n_dense': 4,
    'dense_units': 16
}

print("\nModel Complete.")

print('\nBuilding network 1 - Basic IDS...')


Model Complete.

Building network 1 - Basic IDS...


In [9]:
model1 = create_network(num_classes=num_classes, **network1)

In [10]:
#Creating classifier
classifier = KerasClassifier(model= model1, use_logits=False)


In [11]:
print ("Starting the model fit")

history = classifier.fit(x_train, y_train,
                         batch_size=batch_size,
                         nb_epochs=epochs, verbose=1)

score_model1 = model1.evaluate(x_test,
                               y_test,
                               batch_size=batch_size,
                               verbose=1)

Starting the model fit
Train on 2934817 samples
Epoch 1/10
2934817/2934817 [==============================] - 24s 8us/sample - loss: 0.2694 - accuracy: 0.8930
Epoch 2/10
2934817/2934817 [==============================] - 23s 8us/sample - loss: 0.1767 - accuracy: 0.9306
Epoch 3/10
2934817/2934817 [==============================] - 25s 8us/sample - loss: 0.1638 - accuracy: 0.9351
Epoch 4/10
2934817/2934817 [==============================] - 24s 8us/sample - loss: 0.1554 - accuracy: 0.9384
Epoch 5/10
2934817/2934817 [==============================] - 23s 8us/sample - loss: 0.1500 - accuracy: 0.9404
Epoch 6/10
2934817/2934817 [==============================] - 23s 8us/sample - loss: 0.1459 - accuracy: 0.9418
Epoch 7/10
2934817/2934817 [==============================] - 23s 8us/sample - loss: 0.1427 - accuracy: 0.9427
Epoch 8/10
2934817/2934817 [==============================] - 22s 7us/sample - loss: 0.1405 - accuracy: 0.9434
Epoch 9/10
2934817/2934817 [==============================] - 23

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


In [12]:
print('\nNetwork 1 - Basic IDS results')
print('Hyperparameters:', network1)
print('Test score:', score_model1[0])
print('Test accuracy:', score_model1[1])


Network 1 - Basic IDS results
Hyperparameters: {'n_dense': 4, 'dense_units': 16}
Test score: 0.1006083511940912
Test accuracy: 0.9566052


In [13]:
# Uncomment the attack that you would like to use
attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

# attack = BasicIterativeMethod(estimator=classifier, eps=0.2,verbose=False, max_iter=10)
# x_test_adv = attack.generate(x=x_test)

# attack = ProjectedGradientDescent(estimator=classifier, eps=0.2,verbose=False, max_iter=10)
# x_test_adv = attack.generate(x=x_test)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [1]:
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

NameError: name 'classifier' is not defined

In [15]:
#Prediction Engine
y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(model1.predict(x_test), axis=1)
print(y_true, y_pred)

print("\nCalculating the prediction accuracy ...")
acc = np.sum(y_pred == y_true) / y_test.shape[0]
print("\nTest accuracy of Model: {}%".format(acc * 100))

[1 0 0 ... 0 1 0] [1 0 0 ... 0 1 0]

Calculating the prediction accuracy ...

Test accuracy of Model: 95.66051751044357%


In [16]:
from sklearn.metrics import classification_report, confusion_matrix

print("Generating the confusion matrix ...")
cm = confusion_matrix(y_true, y_pred)
print(cm)

print("Generating the classification report ...")
cr = classification_report(y_true, y_pred)
print(cr)

print("\nOutput Classes in Test Dataset\n{}".format(dataset_test['category'].value_counts()))

Generating the confusion matrix ...
[[367373  16352      0   1584      0]
 [ 11575 317844      0    693      0]
 [     0     70      0     37      0]
 [  1050    464      0  16649      0]
 [     0      5      0      9      0]]
Generating the classification report ...


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      0.95      0.96    385309
           1       0.95      0.96      0.96    330112
           2       0.00      0.00      0.00       107
           3       0.88      0.92      0.90     18163
           4       0.00      0.00      0.00        14

    accuracy                           0.96    733705
   macro avg       0.56      0.57      0.56    733705
weighted avg       0.96      0.96      0.96    733705


Output Classes in Test Dataset
category
DDoS              385309
DoS               330112
Reconnaissance     18163
Normal               107
Theft                 14
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#Confusion matrix and classification report for the attack methods
from sklearn.metrics import classification_report, confusion_matrix

print("Generating the confusion matrix ...")
cm = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print(cm)

print("Generating the classification report ...")
cr = classification_report(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print(cr)

print("\nOutput Classes in Test Dataset\n{}".format(dataset_test['category'].value_counts()))


Generating the confusion matrix ...
[[135242 223448      0  26619      0]
 [127348 170987      0  31777      0]
 [     8     44      0     55      0]
 [ 10691   3664      0   3808      0]
 [     0     14      0      0      0]]
Generating the classification report ...


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.49      0.35      0.41    385309
           1       0.43      0.52      0.47    330112
           2       0.00      0.00      0.00       107
           3       0.06      0.21      0.09     18163
           4       0.00      0.00      0.00        14

    accuracy                           0.42    733705
   macro avg       0.20      0.22      0.19    733705
weighted avg       0.45      0.42      0.43    733705


Output Classes in Test Dataset
category
DDoS              385309
DoS               330112
Reconnaissance     18163
Normal               107
Theft                 14
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
